In [1]:
!pip install autoviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.3
    Uninstalling xgboost-2.1.3:
      Successfully uninstalled xgboost-2.1.3


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('ASPR_Treatments_Locator_20250124.csv')
data.head(5)

,Provider Name,Address 1,Address 2,City,State,Zip,Public Phone,Latitude,Longitude,Geopoint,...,Has Veklury,Has Oseltamivir Generic,Has Oseltamivir Suspension,Has Oseltamivir Tamiflu,Has Baloxavir,Has Zanamivir,Has Peramivir,Grantee Code,Is Flu,Is COVID-19
0,CVS PHARMACY #05246,110 HWY 98,NaN,PORT SAINT JOE,FL,32456,NaN,30.016430,-85.430510,POINT (-85.43051 30.01643),...,False,False,False,False,False,False,False,CV1,False,True
1,PUBLIX PHARMACY #0567,2517 SOUTH FEDERAL HIGHWAY,NaN,FT PIERCE,FL,34982,NaN,27.319630,-80.316317,POINT (-80.316317 27.31963),...,False,False,False,False,False,False,False,PX1,False,True
2,WALGREENS #4379,4320 26TH ST W,NaN,BRADENTON,FL,34205-3563,941-755-8596,27.462954,-82.588373,POINT (-82.588373 27.462954),...,False,True,False,False,False,False,False,WG1,True,True
3,PUBLIX PHARMACY #0894,6820 OKEECHOBEE BLVD,NaN,WEST PALM BEACH,FL,33411,NaN,26.705894,-80.150789,POINT (-80.150789 26.705894),...,False,False,False,False,False,False,False,PX1,False,True
4,PUBLIX PHARMACY #0019,731 DUVAL STATION RD SUITE 4,NaN,JACKSONVILLE,FL,32218,NaN,30.485316,-81.604614,POINT (-81.604614 30.485316),...,False,False,False,False,False,False,False,PX1,False,True


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43806 entries, 0 to 43805
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Provider Name                   43806 non-null  object 
 1   Address 1                       43805 non-null  object 
 2   Address 2                       4873 non-null   object 
 3   City                            43805 non-null  object 
 4   State                           43805 non-null  object 
 5   Zip                             43805 non-null  object 
 6   Public Phone                    13644 non-null  object 
 7   Latitude                        43806 non-null  float64
 8   Longitude                       43806 non-null  float64
 9   Geopoint                        43806 non-null  object 
 10  Last Report Date                43806 non-null  object 
 11  Is PAP Site                     43806 non-null  bool   
 12  Prescribing Services Available  

In [5]:
data.isna().sum().sort_values()

,0
Provider Name,0
Has Peramivir,0
Has Zanamivir,0
Has Baloxavir,0
Has Oseltamivir Tamiflu,0
Has Oseltamivir Suspension,0
Has Oseltamivir Generic,0
Has Veklury,0
Has USG Lagevrio,0
Has Commercial Lagevrio,0


In [6]:
na_feature = [feature for feature in data.columns if data[feature].isna().sum()>1]

for feature in na_feature:
    print(f"{feature}:{data[feature].isna().mean()*100:.3f}%missing value")

Address 2:88.876%missing value
Public Phone:68.854%missing value
Prescribing Services Available:98.637%missing value
Appointment URL:99.260%missing value
Grantee Code:30.834%missing value


In [7]:
data.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Provider Name,43806,39428,HEB PHARMACY,186,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Address 1,43805,42560,101 N MAIN ST,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Address 2,4873,1529,CVS Pharmacy,1416,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,43805,7679,BROOKLYN,560,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State,43805,55,CA,4108,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zip,43805,31890,11214,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Public Phone,13644,13610,559-646-6618,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Latitude,43806.0,NaN,NaN,NaN,36.993039,5.598821,13.478188,33.625985,37.974111,40.875303,64.85082
Longitude,43806.0,NaN,NaN,NaN,-89.938539,15.91725,-159.672639,-96.799787,-85.301666,-78.85553,145.728316
Geopoint,43806,42797,POINT (-66.946804 18.291126),6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# numerical feature

numerical_feature = [feature for feature in data.columns if data[feature].dtypes != 'object']
print("no. of numerical column: ",len(numerical_feature))
data[numerical_feature].head()

no. of numerical column:  21


,Latitude,Longitude,Is PAP Site,Home Delivery,Is ICATT Site,Has USG Product,Has Commercial Product,Has Paxlovid,Has Commercial Paxlovid,Has USG Paxlovid,...,Has Commercial Lagevrio,Has USG Lagevrio,Has Veklury,Has Oseltamivir Generic,Has Oseltamivir Suspension,Has Oseltamivir Tamiflu,Has Baloxavir,Has Zanamivir,Has Peramivir,Is Flu
0,30.016430,-85.430510,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,27.319630,-80.316317,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,27.462954,-82.588373,True,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,True
3,26.705894,-80.150789,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,30.485316,-81.604614,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
# Categorical Column

categorical_feature = [feature for feature in data.columns if data[feature].dtypes == "object"]
print("no. of categorical column: ",len(categorical_feature))
data[categorical_feature].head()

no. of categorical column:  13


,Provider Name,Address 1,Address 2,City,State,Zip,Public Phone,Geopoint,Last Report Date,Prescribing Services Available,Appointment URL,Grantee Code,Is COVID-19
0,CVS PHARMACY #05246,110 HWY 98,NaN,PORT SAINT JOE,FL,32456,NaN,POINT (-85.43051 30.01643),01/17/2025 12:00:00 AM,NaN,NaN,CV1,True
1,PUBLIX PHARMACY #0567,2517 SOUTH FEDERAL HIGHWAY,NaN,FT PIERCE,FL,34982,NaN,POINT (-80.316317 27.31963),01/17/2025 12:00:00 AM,NaN,NaN,PX1,True
2,WALGREENS #4379,4320 26TH ST W,NaN,BRADENTON,FL,34205-3563,941-755-8596,POINT (-82.588373 27.462954),01/17/2025 12:00:00 AM,NaN,NaN,WG1,True
3,PUBLIX PHARMACY #0894,6820 OKEECHOBEE BLVD,NaN,WEST PALM BEACH,FL,33411,NaN,POINT (-80.150789 26.705894),01/17/2025 12:00:00 AM,NaN,NaN,PX1,True
4,PUBLIX PHARMACY #0019,731 DUVAL STATION RD SUITE 4,NaN,JACKSONVILLE,FL,32218,NaN,POINT (-81.604614 30.485316),01/17/2025 12:00:00 AM,NaN,NaN,PX1,True


In [10]:
from autoviz import AutoViz_Class
AV = AutoViz_Class()

Imported v0.1.905. Please call AutoViz in this sequence:
    AV = AutoViz_Class()
    %matplotlib inline
    dfte = AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=1, lowess=False,
               chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30, save_plot_dir=None)


In [11]:
dft = AV.AutoViz("ASPR_Treatments_Locator_20250124.csv")

Shape of your Data Set loaded: (65848, 34)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    Number of Numeric Columns =  2
    Number of Integer-Categorical Columns =  0
    Number of String-Categorical Columns =  3
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  18
    Number of Numeric-Boolean Columns =  0
    Number of Discrete String Columns =  1
    Number of NLP String Columns =  6
    Number of Date Time Columns =  0
    Number of ID Columns =  0
    Number of Columns to Delete =  4
    34 Predictors classified...
        4 variable(s) removed since they were ID or low-information variables
        List of variables removed: ['Prescribing Services Available', 'Appointment URL', 'Has Zanamiv

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
Provider Name,object,0.000000,88,,,No issue
Address 1,object,0.001520,95,,,"1 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
Address 2,object,88.791242,3,,,"58398 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: float, object,"
City,object,0.001520,13,,,"1 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
State,object,0.001520,0,,,"1 missing values. Impute them with mean, median, mode, or a constant value such as 123., 23 rare categories: Too many to list. Group them into a single category or drop the categories., Mixed dtypes: has 2 different data types: object, float,"
Zip,object,0.001520,67,,,"1 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
Public Phone,object,68.741067,31,,,"45211 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: float, object,"
Latitude,float64,0.000000,NA,13.475204,71.297375,Column has 1243 outliers greater than upper bound (51.72) or lower than lower bound(22.78). Cap them or remove them.
Longitude,float64,0.000000,NA,-159.672639,145.728316,Column has 442 outliers greater than upper bound (-51.91) or lower than lower bound(-123.73). Cap them or remove them.
Geopoint,object,0.000000,96,,,63661 rare categories: Too many to list. Group them into a single category or drop the categories.


Number of All Scatter Plots = 3


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt


**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Could not draw wordcloud plot for Address 1. 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/Text

In [12]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [13]:
dft = AV.AutoViz("ASPR_Treatments_Locator_20250124.csv")

Shape of your Data Set loaded: (65848, 34)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    Number of Numeric Columns =  2
    Number of Integer-Categorical Columns =  0
    Number of String-Categorical Columns =  3
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  18
    Number of Numeric-Boolean Columns =  0
    Number of Discrete String Columns =  1
    Number of NLP String Columns =  6
    Number of Date Time Columns =  0
    Number of ID Columns =  0
    Number of Columns to Delete =  4
    34 Predictors classified...
        4 variable(s) removed since they were ID or low-information variables
        List of variables removed: ['Prescribing Services Available', 'Appointment URL', 'Has Zanamiv

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
Provider Name,object,0.000000,88,,,No issue
Address 1,object,0.001520,95,,,"1 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
Address 2,object,88.791242,3,,,"58398 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: float, object,"
City,object,0.001520,13,,,"1 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
State,object,0.001520,0,,,"1 missing values. Impute them with mean, median, mode, or a constant value such as 123., 23 rare categories: Too many to list. Group them into a single category or drop the categories., Mixed dtypes: has 2 different data types: object, float,"
Zip,object,0.001520,67,,,"1 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
Public Phone,object,68.741067,31,,,"45211 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: float, object,"
Latitude,float64,0.000000,NA,13.475204,71.297375,Column has 1243 outliers greater than upper bound (51.72) or lower than lower bound(22.78). Cap them or remove them.
Longitude,float64,0.000000,NA,-159.672639,145.728316,Column has 442 outliers greater than upper bound (-51.91) or lower than lower bound(-123.73). Cap them or remove them.
Geopoint,object,0.000000,96,,,63661 rare categories: Too many to list. Group them into a single category or drop the categories.


Number of All Scatter Plots = 3


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

All Plots done
Time to run AutoViz = 150 seconds 

 ###################### AUTO VISUALIZATION Completed ########################
